<a href="https://colab.research.google.com/github/kyungeunvoyage/HERS_Radar/blob/main/220209_ke_radar_filter_zeropadding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scipy.cluster.hierarchy as sch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cmath as math
import sys
from sklearn.neighbors import NearestCentroid

import imageio
import pathlib
from tqdm import tqdm
import glob
import os

## 1. read data

In [ ]:
def get_pcl_data_(file_path):
    file = open(file_path, "r")
    radar_pcls = file.readlines()
    cnt = 0
    frame = 0
    pcl = 0
    pcl_data = []
    x_data = []
    y_data = []
    z_data = []
    doppler_data = []
    intensity_data = []

    total_x = []
    total_y = []
    total_z = []
    total_dop = []
    total_int  = []

    for i in range(7, len(radar_pcls)):
        if 'x:' in radar_pcls[i]:
            x_int = radar_pcls[i][3:-1]
            x_int_data = float(x_int)
            x_data.append(x_int_data)

        elif 'y:' in radar_pcls[i] and 'velocity:' not in radar_pcls[i] and 'intensity:' not in radar_pcls[i]:
            y_int = radar_pcls[i][3:-1]
            y_int_data = float(y_int)
            y_data.append(y_int_data)
            
        elif 'z:' in radar_pcls[i]:
            z_int = radar_pcls[i][3:-1]
            z_int_data = float(z_int)
            z_data.append(z_int_data)
            
        elif 'doppler_bin:' in radar_pcls[i]:
            doppler_data.append(radar_pcls[i][13:-1])    
            
        elif 'intensity:' in radar_pcls[i]:
            intensity_data.append(radar_pcls[i][11:-1]) 
            
        elif(radar_pcls[i] == 'point_id: 0\n'):
            total_x.append(x_data)  
            total_y.append(y_data)
            total_z.append(z_data)
            total_dop.append(doppler_data)
            total_int.append(intensity_data)
            x_data = []
            y_data = []
            z_data = []
            doppler_data = []
            intensity_data = []
            
    if(len(total_x)>=400):
        total_x = total_x[0:400]
        total_y = total_y[0:400]
        total_z = total_z[0:400]

    else:    
        while (len(total_x) < 400):
            total_x.append(total_x[-1])
            total_y.append(total_x[-1])
            total_z.append(total_x[-1])
    total_x = np.array(total_x, dtype='object')
    total_y = np.array(total_y, dtype='object')
    total_z = np.array(total_z, dtype='object') 
    
#     total_x = np.array(total_x)
#     total_y = np.array(total_y)
#     total_z = np.array(total_z)
#     total_dop = np.array(total_dop)
#     total_int = np.array(total_int)

    return total_x, total_y, total_z

In [ ]:
total_x, total_y, total_z = get_pcl_data_('C:\\Users\\user\\Desktop\\2022_1학기\\0. ITRC\\2. 코드\\dataset\\Training\\1_Running\\radar\\F_001_A1_001.txt')

In [ ]:
def samplingData(total_x, total_y, total_z, num_points):
    total_xyz = []
    for i in range(len(total_x)):
        xyz = np.stack((total_x[i], total_y[i], total_z[i]), axis=1)
        # zero padding the shortage of point cloud datas 
        while(len(xyz) < num_points): 
            num_clusters = num_points - len(xyz)
            while num_clusters > len(xyz):
                num_clusters //= 2
            if num_clusters == 1: # Nearest centroid classifier doesn't offer single class
                xyz = np.append(xyz, np.mean(xyz, axis=0).reshape(-1,3),axis=0)
            else:
                cluster = AgglomerativeClustering(n_clusters=num_clusters, affinity='euclidean', linkage='ward')
                clf = NearestCentroid()
                y_predict = cluster.fit_predict(xyz)
                clf.fit(xyz, y_predict)
                xyz = np.append(xyz, clf.centroids_, axis=0)
        if len(xyz > num_points): # downsample, K-means algorithm, now just cutting off surpluses
            xyz = xyz[0:num_points]
            pass
        else: # len(xyz) is equal to num_points
            pass

        # stack all xyzs to total xyz
        total_xyz.append(xyz)

    total_xyz = np.array(total_xyz)
    return total_xyz


In [ ]:
def SamplingData2(total_x, total_y, total_z, num_points, aa, fixed_length =5, padding_value = 0)
    total_xyz = []
    for i in range(len(total_x)):
        xyz =  np.stack((total_x[i], total_y[i], total_z[i]), axis=1)
        while (len(xyz) < num_points):
            for a in aa:
                total_xyz.append(np.pad(a,(0,fixed_length), 'constant', constant_values=padding_value) [:fixed_length])
            total_xyz = np.concatenate(total_xyz, axis=0).reshape(-1, fixed_length)
            total_xyz = np.array(total_xyz)
        if len(xyz > num_points): # downsample, K-means algorithm, now just cutting off surpluses
            xyz = xyz[0:num_points]
            pass
        else: # len(xyz) is equal to num_points
            pass
    return total_xyz

    
    

In [ ]:
total_xyz = samplingData(total_x, total_y, total_z, 25)
print(total_xyz.shape)

(400, 25, 3)


In [ ]:
total_xyz = SamplingData2(total_x, total_y, total_z, 25,xyz,fixed_length =5, padding_value = 0)
print(total_xyz.shape)

## 2. 전체 Training 파일로 테스트

In [ ]:
parent_dir = 'C:\\Users\\user\\Desktop\\2022_1학기\\0. ITRC\\2. 코드\\dataset\\Training'
sub_dirs=['1_Running','2_Jumping','3_Sit_down_and_stand_up','4_Both_upper_limb_extension','5_falling_forward','6_Right_limb_extension','7_Left_limb_extension']
file_ext='*.txt'

In [ ]:
radar_list = []
index = 0

for sub_dir in sub_dirs:
    files = (os.path.join(parent_dir,sub_dir)) 
    files_2 = (os.path.join(files,'radar'))
    rad_list = os.listdir(files_2) 
    
    cnt = 0
    for rad_element in range(len(rad_list)):
        rad_file = (os.path.join(files_2,rad_list[rad_element]))
        print(rad_file)
        total_x, total_y, total_z = get_pcl_data_(rad_file)
        total_xyz = samplingData(total_x, total_y, total_z, 25)
        
        x_preprocessing = []
        y_preprocessing = []
        z_preprocessing = []

        total_data_= []
        
        for i in range(len(total_xyz)):
            x_ = []
            y_ = []
            z_ = []
            for j in range(len(total_xyz[i][:,0])):
                x_.append([total_xyz[i][:,0][j], 0.0])
                y_.append([total_xyz[i][:,1][j], 0.0])
                z_.append([total_xyz[i][:,2][j], 0.0])
            x_preprocessing.append(x_)   
            x_preprocessing_arr = np.array(x_preprocessing)
            y_preprocessing.append(y_)   
            y_preprocessing_arr = np.array(y_preprocessing)
            z_preprocessing.append(z_)   
            z_preprocessing_arr = np.array(z_preprocessing)
            cnt +=1
        total_data_.append([x_preprocessing ,y_preprocessing,  z_preprocessing])    
        total_data_ = np.array(total_data_)
        print(total_data_.shape) 
            
#         print(cnt)
        if cnt == 400:
            final_data = total_data_
        elif cnt > 400:
            
            final_data = np.concatenate((final_data, total_data_), axis = 0)
        
        print("index:",index)
        print("final_data.shape:", final_data.shape) 
        print("====================================================")    
        index +=1
    
    print("sub_dir:",sub_dir)
    if sub_dir == '1_Running':
        total_final_data = final_data 
        
    elif sub_dir != '1_Running':
        total_final_data = np.concatenate((total_final_data, final_data ), axis = 0)
    print("total_final_data.shape:", total_final_data.shape)    

C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\F_001_A1_001.txt
(1, 3, 400, 25, 2)
index: 0
final_data.shape: (1, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\F_001_A1_002.txt
(1, 3, 400, 25, 2)
index: 1
final_data.shape: (2, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\F_001_A1_003.txt
(1, 3, 400, 25, 2)
index: 2
final_data.shape: (3, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\F_001_A1_004.txt
(1, 3, 400, 25, 2)
index: 3
final_data.shape: (4, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\F_001_A1_005.txt
(1, 3, 400, 25, 2)
index: 4
final_data.shape: (5, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\F_002_A1_001.txt
(1, 3, 400, 25, 2)
index: 5
final_data.shape: (6, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2

(1, 3, 400, 25, 2)
index: 38
final_data.shape: (39, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_003_A1_003.txt
(1, 3, 400, 25, 2)
index: 39
final_data.shape: (40, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_003_A1_004.txt
(1, 3, 400, 25, 2)
index: 40
final_data.shape: (41, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_003_A1_005.txt
(1, 3, 400, 25, 2)
index: 41
final_data.shape: (42, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_004_A1_001.txt
(1, 3, 400, 25, 2)
index: 42
final_data.shape: (43, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_004_A1_002.txt
(1, 3, 400, 25, 2)
index: 43
final_data.shape: (44, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_004_A1_003.txt
(1, 3, 400, 25, 2)
index: 44

(1, 3, 400, 25, 2)
index: 77
final_data.shape: (78, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_010_A1_001.txt
(1, 3, 400, 25, 2)
index: 78
final_data.shape: (79, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_010_A1_002.txt
(1, 3, 400, 25, 2)
index: 79
final_data.shape: (80, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_010_A1_003.txt
(1, 3, 400, 25, 2)
index: 80
final_data.shape: (81, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_010_A1_004.txt
(1, 3, 400, 25, 2)
index: 81
final_data.shape: (82, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_010_A1_005.txt
(1, 3, 400, 25, 2)
index: 82
final_data.shape: (83, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\1_Running\radar\M_010_A1_006.txt
(1, 3, 400, 25, 2)
index: 83

(1, 3, 400, 25, 2)
index: 115
final_data.shape: (24, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\F_005_A2_001.txt
(1, 3, 400, 25, 2)
index: 116
final_data.shape: (25, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\F_005_A2_002.txt
(1, 3, 400, 25, 2)
index: 117
final_data.shape: (26, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\F_005_A2_003.txt
(1, 3, 400, 25, 2)
index: 118
final_data.shape: (27, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\F_005_A2_004.txt
(1, 3, 400, 25, 2)
index: 119
final_data.shape: (28, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\F_005_A2_005.txt
(1, 3, 400, 25, 2)
index: 120
final_data.shape: (29, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\F_005_A2_006.txt
(1, 3, 400, 25, 2)
ind

(1, 3, 400, 25, 2)
index: 153
final_data.shape: (62, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\M_007_A2_006.txt
(1, 3, 400, 25, 2)
index: 154
final_data.shape: (63, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\M_007_A2_007.txt
(1, 3, 400, 25, 2)
index: 155
final_data.shape: (64, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\M_008_A2_001.txt
(1, 3, 400, 25, 2)
index: 156
final_data.shape: (65, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\M_008_A2_002.txt
(1, 3, 400, 25, 2)
index: 157
final_data.shape: (66, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\M_008_A2_003.txt
(1, 3, 400, 25, 2)
index: 158
final_data.shape: (67, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\2_Jumping\radar\M_008_A2_004.txt
(1, 3, 400, 25, 2)
ind

...

(1, 3, 400, 25, 2)
index: 479
final_data.shape: (22, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\6_Right_limb_extension\radar\F_004_A6_005.txt
(1, 3, 400, 25, 2)
index: 480
final_data.shape: (23, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\6_Right_limb_extension\radar\F_004_A6_006.txt
(1, 3, 400, 25, 2)
index: 481
final_data.shape: (24, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\6_Right_limb_extension\radar\F_005_A6_001.txt
(1, 3, 400, 25, 2)
index: 482
final_data.shape: (25, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\6_Right_limb_extension\radar\F_005_A6_002.txt
(1, 3, 400, 25, 2)
index: 483
final_data.shape: (26, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\6_Right_limb_extension\radar\F_005_A6_003.txt
(1, 3, 400, 25, 2)
index: 484
final_data.shape: (27, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset

(1, 3, 400, 25, 2)
index: 515
final_data.shape: (58, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\6_Right_limb_extension\radar\M_007_A6_001.txt
(1, 3, 400, 25, 2)
index: 516
final_data.shape: (59, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\6_Right_limb_extension\radar\M_007_A6_002.txt
(1, 3, 400, 25, 2)
index: 517
final_data.shape: (60, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\6_Right_limb_extension\radar\M_007_A6_003.txt
(1, 3, 400, 25, 2)
index: 518
final_data.shape: (61, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\6_Right_limb_extension\radar\M_007_A6_004.txt
(1, 3, 400, 25, 2)
index: 519
final_data.shape: (62, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\6_Right_limb_extension\radar\M_007_A6_005.txt
(1, 3, 400, 25, 2)
index: 520
final_data.shape: (63, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset

(1, 3, 400, 25, 2)
index: 551
final_data.shape: (2, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\F_001_A7_003.txt
(1, 3, 400, 25, 2)
index: 552
final_data.shape: (3, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\F_001_A7_004.txt
(1, 3, 400, 25, 2)
index: 553
final_data.shape: (4, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\F_001_A7_005.txt
(1, 3, 400, 25, 2)
index: 554
final_data.shape: (5, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\F_002_A7_001.txt
(1, 3, 400, 25, 2)
index: 555
final_data.shape: (6, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\F_002_A7_002.txt
(1, 3, 400, 25, 2)
index: 556
final_data.shape: (7, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7

(1, 3, 400, 25, 2)
index: 587
final_data.shape: (38, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\M_003_A7_002.txt
(1, 3, 400, 25, 2)
index: 588
final_data.shape: (39, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\M_003_A7_003.txt
(1, 3, 400, 25, 2)
index: 589
final_data.shape: (40, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\M_003_A7_004.txt
(1, 3, 400, 25, 2)
index: 590
final_data.shape: (41, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\M_003_A7_005.txt
(1, 3, 400, 25, 2)
index: 591
final_data.shape: (42, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\M_004_A7_001.txt
(1, 3, 400, 25, 2)
index: 592
final_data.shape: (43, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Trai

(1, 3, 400, 25, 2)
index: 623
final_data.shape: (74, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\M_009_A7_003.txt
(1, 3, 400, 25, 2)
index: 624
final_data.shape: (75, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\M_009_A7_004.txt
(1, 3, 400, 25, 2)
index: 625
final_data.shape: (76, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\M_009_A7_005.txt
(1, 3, 400, 25, 2)
index: 626
final_data.shape: (77, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\M_009_A7_006.txt
(1, 3, 400, 25, 2)
index: 627
final_data.shape: (78, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Training\7_Left_limb_extension\radar\M_010_A7_001.txt
(1, 3, 400, 25, 2)
index: 628
final_data.shape: (79, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Trai

In [ ]:
np.save('dgupcl_train.npy', total_final_data)

In [ ]:
dgupcl_train = np.load('dgupcl_train.npy') 
dgupcl_train.shape

(642, 3, 400, 25, 2)

In [ ]:
first = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
print(len(first))

92


In [ ]:
secon = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
print(len(secon))

91


In [ ]:
third = [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]
print(len(third))

91


In [ ]:
forth= [3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3]
print(len(forth))

92


In [ ]:
fif = [4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4]
print(len(fif))

92


In [ ]:
six = [5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5]
print(len(six))

92


In [ ]:
sev = [6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6]
print(len(sev))

92


In [ ]:
test = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6]

In [ ]:
len(test)

642

In [ ]:
import pickle


with open('dgupcl_train_label.pkl', 'wb') as f:
    pickle.dump(test, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('dgupcl_train_label.pkl', 'rb') as f:
    train_pcl_label = pickle.load(f)

In [ ]:
len(train_pcl_label)

642

## 3. 전체 Test 파일로 테스트

In [ ]:
parent_dir = 'C:\\Users\\user\\Desktop\\2022_1학기\\0. ITRC\\2. 코드\\dataset\\Test'
sub_dirs=['1_Running','2_Jumping','3_Sit_down_and_stand_up','4_Both_upper_limb_extension','5_falling_forward','6_Right_limb_extension','7_Left_limb_extension']
file_ext='*.txt'

In [ ]:
radar_list = []
index = 0

for sub_dir in sub_dirs:
    files = (os.path.join(parent_dir,sub_dir)) 
    files_2 = (os.path.join(files,'radar'))
    rad_list = os.listdir(files_2) 
    
    cnt = 0
    for rad_element in range(len(rad_list)):
        rad_file = (os.path.join(files_2,rad_list[rad_element]))
        print(rad_file)
        total_x, total_y, total_z = get_pcl_data_(rad_file)
        total_xyz = samplingData(total_x, total_y, total_z, 25)
        
        x_preprocessing = []
        y_preprocessing = []
        z_preprocessing = []

        total_data_= []
        
        for i in range(len(total_xyz)):
            x_ = []
            y_ = []
            z_ = []
            for j in range(len(total_xyz[i][:,0])):
                x_.append([total_xyz[i][:,0][j], 0.0])
                y_.append([total_xyz[i][:,1][j], 0.0])
                z_.append([total_xyz[i][:,2][j], 0.0])
            x_preprocessing.append(x_)   
            x_preprocessing_arr = np.array(x_preprocessing)
            y_preprocessing.append(y_)   
            y_preprocessing_arr = np.array(y_preprocessing)
            z_preprocessing.append(z_)   
            z_preprocessing_arr = np.array(z_preprocessing)
            cnt +=1
        total_data_.append([x_preprocessing ,y_preprocessing,  z_preprocessing])    
        total_data_ = np.array(total_data_)
        print(total_data_.shape) 
            
#         print(cnt)
        if cnt == 400:
            final_data = total_data_
        elif cnt > 400:
            
            final_data = np.concatenate((final_data, total_data_), axis = 0)
        
        print("index:",index)
        print("final_data.shape:", final_data.shape) 
        print("====================================================")    
        index +=1
    
    print("sub_dir:",sub_dir)
    if sub_dir == '1_Running':
        total_final_data = final_data 
        
    elif sub_dir != '1_Running':
        total_final_data = np.concatenate((total_final_data, final_data ), axis = 0)
    print("total_final_data.shape:", total_final_data.shape)    

C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\1_Running\radar\F_006_A1_001.txt
(1, 3, 400, 25, 2)
index: 0
final_data.shape: (1, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\1_Running\radar\F_006_A1_002.txt
(1, 3, 400, 25, 2)
index: 1
final_data.shape: (2, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\1_Running\radar\F_006_A1_003.txt
(1, 3, 400, 25, 2)
index: 2
final_data.shape: (3, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\1_Running\radar\F_006_A1_004.txt
(1, 3, 400, 25, 2)
index: 3
final_data.shape: (4, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\1_Running\radar\F_006_A1_005.txt
(1, 3, 400, 25, 2)
index: 4
final_data.shape: (5, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\1_Running\radar\F_006_A1_006.txt
(1, 3, 400, 25, 2)
index: 5
final_data.shape: (6, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\1_Runn

(1, 3, 400, 25, 2)
index: 39
final_data.shape: (17, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\2_Jumping\radar\M_014_A2_001.txt
(1, 3, 400, 25, 2)
index: 40
final_data.shape: (18, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\2_Jumping\radar\M_014_A2_002.txt
(1, 3, 400, 25, 2)
index: 41
final_data.shape: (19, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\2_Jumping\radar\M_014_A2_003.txt
(1, 3, 400, 25, 2)
index: 42
final_data.shape: (20, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\2_Jumping\radar\M_014_A2_004.txt
(1, 3, 400, 25, 2)
index: 43
final_data.shape: (21, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\2_Jumping\radar\M_014_A2_005.txt
(1, 3, 400, 25, 2)
index: 44
final_data.shape: (22, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\2_Jumping\radar\M_014_A2_006.txt
(1, 3, 400, 25, 2)
index: 45
final_data.shape: (23, 

(1, 3, 400, 25, 2)
index: 76
final_data.shape: (8, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\4_Both_upper_limb_extension\radar\M_012_A4_005.txt
(1, 3, 400, 25, 2)
index: 77
final_data.shape: (9, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\4_Both_upper_limb_extension\radar\M_012_A4_006.txt
(1, 3, 400, 25, 2)
index: 78
final_data.shape: (10, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\4_Both_upper_limb_extension\radar\M_013_A4_001.txt
(1, 3, 400, 25, 2)
index: 79
final_data.shape: (11, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\4_Both_upper_limb_extension\radar\M_013_A4_002.txt
(1, 3, 400, 25, 2)
index: 80
final_data.shape: (12, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\4_Both_upper_limb_extension\radar\M_013_A4_003.txt
(1, 3, 400, 25, 2)
index: 81
final_data.shape: (13, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Te

(1, 3, 400, 25, 2)
index: 113
final_data.shape: (22, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\5_falling_forward\radar\M_014_A5_006.txt
(1, 3, 400, 25, 2)
index: 114
final_data.shape: (23, 3, 400, 25, 2)
sub_dir: 5_falling_forward
total_final_data.shape: (115, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\6_Right_limb_extension\radar\F_006_A6_001.txt
(1, 3, 400, 25, 2)
index: 115
final_data.shape: (1, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\6_Right_limb_extension\radar\F_006_A6_002.txt
(1, 3, 400, 25, 2)
index: 116
final_data.shape: (2, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\6_Right_limb_extension\radar\F_006_A6_003.txt
(1, 3, 400, 25, 2)
index: 117
final_data.shape: (3, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\6_Right_limb_extension\radar\F_006_A6_004.txt
(1, 3, 400, 25, 2)
index: 118
final_data.shape: (4, 3, 400, 25, 2)
C:\Users\

(1, 3, 400, 25, 2)
index: 149
final_data.shape: (12, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\7_Left_limb_extension\radar\M_013_A7_003.txt
(1, 3, 400, 25, 2)
index: 150
final_data.shape: (13, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\7_Left_limb_extension\radar\M_013_A7_004.txt
(1, 3, 400, 25, 2)
index: 151
final_data.shape: (14, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\7_Left_limb_extension\radar\M_013_A7_005.txt
(1, 3, 400, 25, 2)
index: 152
final_data.shape: (15, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\7_Left_limb_extension\radar\M_013_A7_006.txt
(1, 3, 400, 25, 2)
index: 153
final_data.shape: (16, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\7_Left_limb_extension\radar\M_013_A7_007.txt
(1, 3, 400, 25, 2)
index: 154
final_data.shape: (17, 3, 400, 25, 2)
C:\Users\user\Desktop\2022_1학기\0. ITRC\2. 코드\dataset\Test\7_Left_limb_extensi

In [ ]:
np.save('dgupcl_test.npy', total_final_data)

In [ ]:
dgupcl_test = np.load('dgupcl_test.npy') 
dgupcl_test.shape

(161, 3, 400, 25, 2)

In [ ]:
test_la = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,]

In [ ]:
with open('dgupcl_test_label.pkl', 'wb') as f:
    pickle.dump(test_la, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('dgupcl_test_label.pkl', 'rb') as f:
    test_pcl_label = pickle.load(f)

In [ ]:
len(test_pcl_label)

161